In [ ]:
using DelimitedFiles,HDF5, PyCall, PyPlot, NRRD,FileIO, TiffImages,Images, ProgressMeter, Statistics,LinearAlgebra, NaNStatistics
using _Data

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport matplotlib.patches as patches
@pyimport scipy.ndimage as ndimage

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
ds_save_cy_ref = Dataset("20220416_123633", "chuyu", gethostname() == "roli-$(5)" ? "/data" : "/nfs/data$(5)")
# load atlas 
atlas_ref_img = TiffImages.load(data_path(ds_save_cy_ref)*"/Elavl3-H2BRFP.tif");
atlas_ref = convert(Array{Float32}, atlas_ref_img);

stack_img_ref = fill(NaN32, size(atlas_ref,2), size(atlas_ref,1), size(atlas_ref,3))
for (i, z) in enumerate(1:size(atlas_ref,3))
    stack_img_ref[:,:,i].= atlas_ref[:,:,z]'
end




In [ ]:
distance_all = zeros(Float32, 11,11,11)
x_index, y_index, z_index = numpy.meshgrid(1:size(distance_all, 1),1:size(distance_all, 2), 1:size(distance_all, 3))
for i in 1:length(x_index)
    x = x_index[i]
    y = y_index[i]
    z = z_index[i]
    distance_all[x, y, z] = norm([x-Int32.(ceil(size(distance_all, 1)/2)), y-Int32.(ceil(size(distance_all, 2)/2)), 2*(z-Int32.(ceil(size(distance_all, 3)/2)))]);
end

kernel = Int32.(collect(distance_all.<=5)); #check this

In [ ]:
@pyimport matplotlib.colors as mpl_colors

In [ ]:
@pyimport matplotlib.cm as cm 

In [ ]:
hot = cm.get_cmap("hot", 100)
alpha_top = 50
cmaplist = [hot(i) for i in 1:(hot.N)]
cmaplist_new = []
length_cmaplist = length(cmaplist)
for (i, color_) in enumerate(cmaplist)
    new_color_ = numpy.copy(color_)
    if i<=alpha_top
        new_color_[4] = (100/alpha_top)*i/length_cmaplist
    end
    append!(cmaplist_new,[new_color_]) 
end
hot_new = mpl_colors.LinearSegmentedColormap.from_list("hot_new",cmaplist_new,100)

In [ ]:
data_all_exp = 
[
    ["20220407_152537", 4, "jen"],
    ["20220406_111526", 9, "jen"],
    ["20220407_090156", 5, "jen"],
    ["20220417_165530", 2, "jen"],
    ["20220406_153842", 9, "jen"],
    ["20220405_171444", 25, "jen"],
    ["20220416_160516", 6, "jen"]
];

In [ ]:
## cell loc
analyzer = "chuyu"
cell_locs_all = []
cell_locs_all_original = []
experiment_filename_all = []
specificity_population_z_all = []
for which_combo in 1:length(data_all_exp)
    data_combo = data_all_exp[which_combo]
    experiment_filename_1 = data_combo[1]
    server_1 = data_combo[2]

    experimenter = data_combo[end]
    ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
    ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

    NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
    NMF_file = h5open(NMF_filename, "r")
    global Z_all = HDF5.readmmap(NMF_file["Z_all"])
    global X_all = HDF5.readmmap(NMF_file["X_all"])
    global Y_all = HDF5.readmmap(NMF_file["Y_all"])
    global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
    close(NMF_file)

    n_neurons = length(X_all)
    cell_locs = fill(NaN32, n_neurons, 3)
    cell_locs[:, 1] = X_all
    cell_locs[:, 2] = Y_all
    cell_locs[:, 3] = Z_all
    append!(cell_locs_all_original,[cell_locs]);
    
    all_files = readdir(data_path(ds_save_cy_1))
    long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
    spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
    candidate_filename = long_name_files[spatial_info_index]
    which_file = [occursin(experiment_filename_1, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
    @assert(length(candidate_filename[which_file]) == 1)
    save_file_name = candidate_filename[which_file][1]
    info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
    file = h5open(info_filename, "r")
    spatial_info_all_1 = HDF5.readmmap(file["spatial_info_all"])
    close(file)
    specificity_population_z_1 = (spatial_info_all_1.-numpy.nanmean(spatial_info_all_1))./numpy.nanstd(spatial_info_all_1);
    append!(specificity_population_z_all,[specificity_population_z_1])
        
    loc_save_path = joinpath(data_path(ds_save_cy_1), "$experiment_filename_1"*"_engert")
    registered_cell_locs_original = readdlm(joinpath(loc_save_path, experiment_filename_1*"_nmf_loc_engert_registered.txt"));
    registered_cell_locs = registered_cell_locs_original[:,1:3];


    atlas_cell_locs = Int32.(round.(registered_cell_locs));
    atlas_cell_locs[:,1] .+= 329;
    atlas_cell_locs[:,2] .+= 41;

    atlas_cell_locs[:,1] .= size(stack_img_ref,1).-atlas_cell_locs[:,1];
    atlas_cell_locs[:,3] .= size(stack_img_ref,3).-atlas_cell_locs[:,3];

    cell_locs = fill(NaN32, n_neurons, 3)
    @showprogress for (i, which_neuron) in enumerate(1:n_neurons)
        cell_locs[i, :] = mean(atlas_cell_locs[neuron_label.==which_neuron, :], dims=1)

    end
    
    append!(cell_locs_all,[cell_locs]);
    append!(experiment_filename_all,[experiment_filename_1]);
end

In [ ]:
specificity_population_z_threshold = 3

place_cell_index_all = []
@showprogress for i in 1:length(data_all_exp)
    specificity_population_z = specificity_population_z_all[i]
    place_cell_index = intersect(findall(specificity_population_z.>=specificity_population_z_threshold));
    append!(place_cell_index_all,[place_cell_index]) 
end

for place_cell_index in place_cell_index_all
    println(length(place_cell_index))
end


cell_locs_tog = reduce(vcat,cell_locs_all)[:,1:3];
place_cell_locs_all = [cell_locs_all[i][place_cell_index_all[i],:] for i in 1:length(cell_locs_all)]
place_cell_locs_all_original = [cell_locs_all_original[i][place_cell_index_all[i],:] for i in 1:length(cell_locs_all_original)]
place_cell_locs_tog = reduce(vcat,place_cell_locs_all)[:,1:3];

In [ ]:
fig, ax = subplots(1,1,dpi=250, figsize=(15,10))
for (i,cell_locs) in enumerate(cell_locs_all)
    place_cell_index = Int32.(place_cell_index_all[i])
    ax.scatter(cell_locs[place_cell_index,1], cell_locs[place_cell_index,2], s=2, label=experiment_filename_all[i])
end
ax.set_aspect("equal", "box")
ax.legend()


fig_bg = maximum(stack_img_ref, dims=3)[:,:,1]
ax.imshow(fig_bg', cmap="gray")
# axvline(620)
# axvline(450)

# fig.savefig(joinpath(fig_path, "distribution_anatomy_1_individual_fish.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)


In [ ]:
spatial_p_img = zeros(Int32, size(stack_img_ref));
for i in 1:size(place_cell_locs_tog,1)
    x = Int32.(numpy.round(place_cell_locs_tog[i,1]))
    y = Int32.(numpy.round(place_cell_locs_tog[i,2]))
    z = Int32.(numpy.round(place_cell_locs_tog[i,3]))
    if x.<=size(spatial_p_img,1) && y.<=size(spatial_p_img,2) && z.<=size(spatial_p_img,3) && x.>0 && y.>0 &&  z.>0
        spatial_p_img[x,y,z] += 1
    end
end

In [ ]:
fig_bg = maximum(spatial_p_img, dims=3)[:,:,1]
fig, ax = subplots(1,1,dpi=250)
ax.imshow(fig_bg', cmap="gray")

In [ ]:
spatial_p_img_convolved = ndimage.convolve(spatial_p_img, kernel);

In [ ]:
# using NRRD

# save(joinpath("/home/chuyu/to_be_registered", "spatial_p_img.nrrd"), spatial_p_img)

In [ ]:
spatial_p_img_copy = Float32.(spatial_p_img_convolved)
# spatial_p_img_copy[spatial_p_img_copy.<3] .= NaN

vmax = maximum(spatial_p_img_copy)
fig, ax = subplots(1,1,dpi=250)
fig_bg = maximum(stack_img_ref, dims=3)[:,:,1]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=2)
ax.imshow(fig_bg', cmap="gray")
sp1 =ax.imshow(spatial_p_bg', cmap=hot_new, vmin=0, vmax=vmax)
# colorbar(sp1)
axis("off")
# fig.savefig(joinpath(fig_path, "distribution_anatomy_1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
# fig.savefig(joinpath(fig_path, "distribution_anatomy_1.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

fig, ax = subplots(1,1,dpi=250)
fig_bg = maximum(stack_img_ref, dims=2)[:,1,:]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=1)
ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
sp2 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
axis("off")
# fig.savefig(joinpath(fig_path, "distribution_anatomy_2.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
# fig.savefig(joinpath(fig_path, "distribution_anatomy_2.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)



fig, ax = subplots(1,1,dpi=250)
fig_bg = maximum(stack_img_ref, dims=1)[1,:,:]
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=0)
ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
sp3 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
axis("off")
# fig.savefig(joinpath(fig_path, "distribution_anatomy_3.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
# fig.savefig(joinpath(fig_path, "distribution_anatomy_3.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)



In [ ]:
fig = figure()
spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=2)
sp1 =imshow(spatial_p_bg', cmap=hot_new, vmin=0, vmax=vmax)
sp1.set_visible(false)
axis("off")
clb = colorbar(shrink=0.3, aspect=8, mappable=sp1, ticks=[0, vmax], fraction=0.999, pad=0)
clb.patch.set_facecolor((0.2, 0.2, 0.2, 1.0))
# clb.ax.tick_params(colors="white") 
# clb.outline.set_color("white")
# fig.savefig(joinpath(fig_path, "distribution_anatomy_colorbar.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure3/spatial_info_place_cell"
mkpath(fig_path)

In [ ]:
spatial_p_img_convolved_all = []

for (which_fig,specificity_population_z_threshold) in enumerate([1,3,5])


    place_cell_index_all = []
    for i in 1:length(data_all_exp)
        specificity_population_z = specificity_population_z_all[i]
        place_cell_index = intersect(findall(specificity_population_z.>=specificity_population_z_threshold));
        append!(place_cell_index_all,[place_cell_index]) 
    end

    for place_cell_index in place_cell_index_all
        println(length(place_cell_index))
    end


    cell_locs_tog = reduce(vcat,cell_locs_all)[:,1:3];
    place_cell_locs_all = [cell_locs_all[i][place_cell_index_all[i],:] for i in 1:length(cell_locs_all)]
    place_cell_locs_all_original = [cell_locs_all_original[i][place_cell_index_all[i],:] for i in 1:length(cell_locs_all_original)]
    place_cell_locs_tog = reduce(vcat,place_cell_locs_all)[:,1:3];

    
    spatial_p_img = zeros(Int32, size(stack_img_ref));
    for i in 1:size(place_cell_locs_tog,1)
        x = Int32.(numpy.round(place_cell_locs_tog[i,1]))
        y = Int32.(numpy.round(place_cell_locs_tog[i,2]))
        z = Int32.(numpy.round(place_cell_locs_tog[i,3]))
        if x.<=size(spatial_p_img,1) && y.<=size(spatial_p_img,2) && z.<=size(spatial_p_img,3) && x.>0 && y.>0 &&  z.>0
            spatial_p_img[x,y,z] += 1
        end
    end
    
    spatial_p_img_convolved = ndimage.convolve(spatial_p_img, kernel);
    
    append!(spatial_p_img_convolved_all, [spatial_p_img_convolved])

end

In [ ]:
stack_img_cropped = copy(stack_img_ref)



stack_img_cropped[1:98,:,:] .= NaN32;
stack_img_cropped[1090:end,:,:] .= NaN32;

stack_img_cropped[:,1:10,:] .= NaN32;
stack_img_cropped[:,615:end,:] .= NaN32;

stack_img_cropped[:,:,1:25] .= NaN32;
stack_img_cropped[:,:,129:end] .= NaN32;


In [ ]:
vmax_all = [20,15,10]
for (which_fig,specificity_population_z_threshold) in enumerate([1,3,5])
# for specificity_population_z_threshold in [1]

    spatial_p_img_convolved = spatial_p_img_convolved_all[which_fig]

    
    spatial_p_img_copy = Float32.(spatial_p_img_convolved)
    
    spatial_p_img_copy[isnan.(stack_img_cropped)] .= NaN32
    # spatial_p_img_copy[spatial_p_img_copy.<3] .= NaN

    vmax = vmax_all[which_fig]
    fig, ax = subplots(1,1,dpi=250)
    fig_bg = nanmaximum(stack_img_cropped, dims=3)[:,:,1]
    spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=2)
    ax.imshow(fig_bg', cmap="gray")
    sp1 =ax.imshow(spatial_p_bg', cmap=hot_new, vmin=0, vmax=vmax)
    # colorbar(sp1)
    axis("off")
    fig.savefig(joinpath(fig_path, "distribution_anatomy_1_info$specificity_population_z_threshold.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)

    fig, ax = subplots(1,1,dpi=250)
    fig_bg = nanmaximum(stack_img_cropped, dims=2)[:,1,:]
    spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=1)
    ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
    sp2 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
    axis("off")
    fig.savefig(joinpath(fig_path, "distribution_anatomy_2_info$specificity_population_z_threshold.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)



    fig, ax = subplots(1,1,dpi=250)
    fig_bg = nanmaximum(stack_img_cropped, dims=1)[1,:,:]
    spatial_p_bg = numpy.nanmax(spatial_p_img_copy, axis=0)
    ax.imshow(fig_bg', cmap="gray",aspect=2/0.798)
    sp3 =ax.imshow(spatial_p_bg', cmap=hot_new,aspect=2/0.798, vmin=0, vmax=vmax)
    axis("off")
    fig.savefig(joinpath(fig_path, "distribution_anatomy_3_info$specificity_population_z_threshold.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)


#     fig = figure(figsize=(2,2))
#     sp1 =imshow(spatial_p_bg', cmap=hot_new, vmin=0, vmax=vmax)

#     sp1.set_visible(false)
#     axis("off")
#     clb = colorbar(shrink=0.3, aspect=8, mappable=sp1, ticks=[0, vmax], fraction=0.999, pad=0)
#     clb.patch.set_facecolor((0.2, 0.2, 0.2, 1.0))
#     clb.ax.tick_params(colors="white", width=1,length=0) 
#     clb.outline.set_color("white")
#     clb.outline.set_linewidth(1)
#     fig.savefig(joinpath(fig_path, "distribution_anatomy_spe$specificity_population_z_threshold colorbar.pdf"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end